# Hypothesentest

Nachfolgend wird der Hypothesentest nach Spearman durchgeführt. Einmal für alle Städte und Jahre und einmal für jede Stadt einzeln. <br>
<br>
Warum Spearman statt Pearson?
- misst monotonen Zusammenhang (nicht nur linear)
- keine Normalverteilung nötig
- robust gegenüber Ausreißern und vielen Nullen
- geeignet für zusammengesetzte / ordinale Variablen

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

konsum = pd.read_csv("../../Daten_sets/wasser_cleaning/alle_staedte2.csv", sep=";")
berichte = pd.read_csv("../../statistik/prozentuale_aenderung_blaulicht_drogen_tf-idf.csv", sep=";", decimal=",")

In [5]:
# Wide-Format → Long-Format
konsum_long = konsum.melt(
    id_vars="Year",            # Zeitvariable behalten
    var_name="Stadt",          # Städtenamen aus Spalten
    value_name="Konsum_mg_tag" # Messwert: Drogenkonsum
)

# Einheitliche Jahresbezeichnung
konsum_long = konsum_long.rename(columns={"Year": "Jahr"})

# Fehlende Konsumwerte entfernen
konsum_long = konsum_long.dropna()

# Städtenamen vereinheitlichen (Merge-Kompatibilität)
konsum_long["Stadt"] = konsum_long["Stadt"].replace({
    "Munich G": "München",
    "Nuremberg": "Nuernberg",
    "Saarbrücken (2)": "Saarbruecken"
})

# Relevante Variablen aus Berichte-Datensatz filtern
berichte_sub = berichte[["Stadt", "Jahr", "Frequenz_Prozent"]].dropna()

# Zusammenführen beider Datensätze (Stadt × Jahr)
df = pd.merge(
    konsum_long,
    berichte_sub,
    on=["Stadt", "Jahr"],
    how="inner"           # nur vollständige Beobachtungen
)

In [8]:
# -------------------------------
# Hypothesentest für den Gesamtendatensatz
# -------------------------------

rho, p = spearmanr(
    df["Konsum_mg_tag"],
    df["Frequenz_Prozent"]
)

# Testergebnisse ausgeben
print(f"Spearman rho = {rho:.3f}")
print(f"p-Wert = {p:.4f}")

# Hypothesenentscheidung mit α = 0.05
if p <= 0.05:
    print("➡ Signifikanter Zusammenhang")
else:
    print("➡ Kein signifikanter Zusammenhang")

Spearman rho = 0.032
p-Wert = 0.8411
➡ Kein signifikanter Zusammenhang


Ergebnis: <br>
<br>
H0 wird verworfen und H1 wird angenommen <br>
-> Es gibt keinen signifikanten Zusammenhang, wenn man den gesamten Datensatz auf einmal betrachtet

In [7]:
# -------------------------------
# Hypothesentest für jede Stadt
# -------------------------------

ergebnisse = []

for stadt in df["Stadt"].unique():

    # Teildatensatz pro Stadt
    df_stadt = df[df["Stadt"] == stadt]

    # Mindestanzahl Beobachtungsdaten
    if len(df_stadt) < 3:
        continue

    # Spearman-Korrelation
    rho, p = spearmanr(
        df_stadt["Konsum_mg_tag"],
        df_stadt["Frequenz_Prozent"]
    )

    # Ergebnisse speichern
    ergebnisse.append({
        "Stadt": stadt,
        "n": len(df_stadt),
        "Spearman_rho": round(rho, 3),
        "p_Wert": round(p, 4),
        "Signifikant (α=0.05)": p <= 0.05
    })

# Ausgabe Ergebnisse
ergebnis_df = pd.DataFrame(ergebnisse)
print(ergebnis_df)


          Stadt  n  Spearman_rho  p_Wert  Signifikant (α=0.05)
0      Chemnitz  7         0.867  0.0115                  True
1      Dortmund  4         0.800  0.2000                 False
2        Erfurt  7         0.847  0.0162                  True
3       München  9        -0.450  0.2242                 False
4     Nuernberg  6         0.600  0.2080                 False
5  Saarbruecken  8        -0.548  0.1600                 False


Interpretation der neuen Ergebnisse:

Chemnitz und Erfurt<br>
→ Signifikanter positiver Zusammenhang<br>
→ Wenn der Konsum steigt, steigt auch die Berichtsrelevanz<br>
→ H₀ verworfen<br>

Dortmund, München, Nürnberg, Saarbrücken<br>
→ Kein signifikanter Zusammenhang<br>
→ Entweder zu wenige Daten oder kein stabiler Trend<br>
→ H₀ beibehalten<br>

Negative rhos (München, Saarbrücken)<br>
→ Deuten auf gegenläufige Trends hin<br>
→ Aber nicht signifikant, daher keine belastbare Aussage<br>

Fazit:<br>
Nur in Chemnitz und Erfurt zeigt sich ein statistisch belegbarer Zusammenhang zwischen Konsum und Berichterstattung. In den übrigen Städten kann kein signifikanter Zusammenhang nachgewiesen werden.